# 青空文庫テキスト-スクレイピングスクリプト

### 環境準備

In [1]:
!pip install beautifulsoup4

### ライブラリのインポート

In [79]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os

### ファイルのダウンロードとzip解凍の必要関数定義

In [80]:
def zip_extract(filename):
    """ファイル名を指定して zip ファイルをカレントディレクトリに展開する
    """
    target_directory = 'data'
    zfile = zipfile.ZipFile(filename)
    zfile.extractall(target_directory)

In [81]:
def download_file(url):
    """URL を指定してカレントディレクトリにファイルをダウンロードする
    """
    filename = 'data/' + url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
    zip_extract(filename)
    os.remove(filename)
    return filename

### URLの定義

In [82]:
base_url = 'https://www.aozora.gr.jp/'  # 青空文庫共通
saku_list = base_url + 'index_pages/person96.html#sakuhin_list_1'  # 作家固有

In [83]:
res = requests.get(saku_list)

In [84]:
#print(res.text)

In [85]:
soup = BeautifulSoup(res.text, "html.parser")

In [86]:
elems = soup.select('li > a')

### ダウンロードするためのURLを集める

In [87]:
new_list = []
for elem in elems:
    url = elem.attrs['href']
    if url[0:6] != "person":
        new_list.append(base_url + elem.attrs['href'][3:])

### スクレイピング（というかzipのダウンロードして解凍）

In [88]:
for url2 in new_list:
    base_url2 = url2[:url2.rfind('/')]
    res2 = requests.get(url2)
    soup2 = BeautifulSoup(res2.text, "html.parser")
    elems2 = soup2.select('table > tr > td > a')
    for elem2 in elems2:
        url3 = elem2.attrs['href']
        if url3[-3:] == 'zip':
            url4 = base_url2 + url3[1:]
            download_file(url4)